In [1]:
import pandas as pd
import mlflow
import mlflow.pyfunc

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from tqdm import tqdm #Progress bar for loops

import logging
logging.getLogger("mlflow").setLevel(logging.ERROR) # Suppress the MLflow warning - It only shows warning below the ERROR level.

In [2]:
# Set up MLflow tracking
experiment_name = "dtc_persona_analysis"
model_name = "dtc_persona_clustering_model"

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='/Users/matthiasmotl/neuefische/repositories/dtc/dtc_persona_analysis/01_model/mlruns/1', creation_time=1750796641317, experiment_id='1', last_update_time=1750796641317, lifecycle_stage='active', name='dtc_persona_analysis', tags={}>

In [3]:
# Enable scikit-learn autologging
mlflow.sklearn.autolog()

In [4]:
# Load the dataset
X = pd.read_csv("../data/test_ref.csv")

In [5]:
# Define the range of clusters to test
k_min = 2
k_max = 15


# Iterate over the range of clusters
for clusters in tqdm(range(k_min, k_max + 1)):
    
    # Start a new MLflow run for each cluster count
    with mlflow.start_run(): #run_name=f"kmeans_basic_k={C}"):

        # Instantiate your model
        # MLflow will capture these parameters automatically
        model = KMeans(n_clusters=clusters, n_init=10)#, random_state=42)
        
        # Fit the model
        # MLflow intercepts this .fit() call to log metrics and artifacts
        model.fit(X)

        # inertia is the sum of squared distances to the nearest cluster center
        # It is a measure of how tightly the clusters are packed
        # Lower inertia means better clustering
        mlflow.log_metric("inertia", model.inertia_)

        # Log silhouette score, which measures how similar an object is to its own cluster compared to other clusters
        # A higher silhouette score indicates better-defined clusters 
        silhouette = silhouette_score(X, model.fit_predict(X))
        mlflow.log_metric("silhouette", silhouette)

        # Log the ratio of silhouette score to inertia
        # This ratio can help assess the quality of clustering relative to the compactness of clusters
        score = silhouette * 1000 / model.inertia_
        mlflow.log_metric("silhouette_inertia_ratio", score)

        # Log the number of clusters
        mlflow.log_param("n_clusters", clusters)

        # Log the model itself
        mlflow.set_tag("run_purpose", "new data test")

100%|██████████| 14/14 [00:59<00:00,  4.24s/it]


#### Registry

In [6]:
# Search for runs in the current experiment to find the best model, sorted by the main score "silhouette_inertia_ratio" descending
experiment = mlflow.get_experiment_by_name(experiment_name)
runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.silhouette_inertia_ratio DESC"],
    max_results=1
)
best_run = runs.iloc[0]
print("Best run ID:", best_run.run_id)
print("Best silhouette_inertia_ratio:", best_run["metrics.silhouette_inertia_ratio"])
print("Count of clusters:", best_run["params.n_clusters"])

Best run ID: 64df73e064eb4f23a0952e2798c82c27
Best silhouette_inertia_ratio: 0.029651298662187457
Count of clusters: 3


In [7]:
# Register model
# This will register the model with the best silhouette_inertia_ratio
model_uri = f"runs:/{best_run.run_id}/model"
mlflow.register_model(model_uri, "dtc_persona_clustering_model")

Registered model 'dtc_persona_clustering_model' already exists. Creating a new version of this model...
Created version '3' of model 'dtc_persona_clustering_model'.


<ModelVersion: aliases=[], creation_timestamp=1752580590524, current_stage='None', description=None, last_updated_timestamp=1752580590524, name='dtc_persona_clustering_model', run_id='64df73e064eb4f23a0952e2798c82c27', run_link=None, source='/Users/matthiasmotl/neuefische/repositories/dtc/dtc_persona_analysis/01_model/mlruns/1/64df73e064eb4f23a0952e2798c82c27/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

#### Cluster Centers

In [8]:
# Define the Model URI
model_version = 2
model_uri = f"models:/{model_name}/{model_version}"

In [9]:
print(f"Loading model from: {model_uri}")

# loading the model using mlflow.sklearn.load_model to ensure compatibility with scikit-learn attributes
try:
    loaded_model = mlflow.sklearn.load_model(model_uri)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    # Exit if model fails to load
    exit()

Loading model from: models:/dtc_persona_clustering_model/2
Model loaded successfully!


In [10]:
# Access the cluster centers from the loaded model
cluster_centers = loaded_model.cluster_centers_
cluster_centers

array([[-9.64147859,  9.55177858,  6.52788896, -5.80578982, -6.44716378,
        -6.09991742, -4.15426424,  0.26307602, -1.21858764, -3.80585895],
       [ 2.18375701, -7.27548384, -4.0674269 , -2.92685785, -1.04996438,
         5.54416059, -6.04791353,  0.39094882,  1.90642709, -9.28955403],
       [-2.3842275 ,  9.04632818,  4.70198909,  2.08257798, -6.66452801,
        -6.95698357, -8.88479977,  7.4740681 ,  2.09595048,  4.44140944]])